# Projeto 1 - Ciência dos Dados

Nome: Arthur Cisotto Machado

Nome: Alessandra Yumi Carvalho Ogawa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import emoji
from emoji import UNICODE_EMOJI

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Arthur\Dropbox\A - ARTHUR\2o Semestre\CDados\P1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [19]:
# https://docs.python.org/3/library/re.html#


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[\\n:;?\"\'().,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_split = text_subbed.split()
    return ' '.join(text_split).lower()

def separa_emoji(tweet):
    '''
    Essa função separa os emojis juntos, além de transcreve-los
    para suas respectivas strings descritivas, optimizando um pouco o processamento.
    Nota-se que alguns emojis não usados não estão no Unicode da língua portuguesa,
    adotamos assim o de língua inglesa como alternativa de transcrição.
    '''
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        elif emoji1 in UNICODE_EMOJI['en']:
            modified[i]=UNICODE_EMOJI['en'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)
        
    return modified

def limpeza_e_emoji(tweet):
    texto_filtrado = cleanup(tweet)
    sem_emoji = separa_emoji(texto_filtrado)
    return sem_emoji

In [4]:
filename = 'la casa de papel.xlsx'

In [20]:
train = pd.read_excel(filename)
train['Treinamento'] = train['Treinamento'].apply(limpeza_e_emoji)
train.head(5)

,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0
1,agora vou assistir lá casa de papel e dormir p...,0.0
2,dedo coçando pra assistir lá casa de papel mas...,0.0
3,la casa de papel me faz torcer pros bandidos,1.0
4,la casa de papel acabou comigo me encontro des...,1.0


In [21]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,"Classificação (relevante = 1, não relevante = 0)"
0,quero terminar de ver a nova parte de la casa ...,0
1,não existe outra série no mundo que mexa comig...,1
2,tá foda de desviar de todos os spoilers de la ...,1
3,@todoroki_jun2 né tô me sentindo em lá casa de...,0
4,la casa de papel é tão ruim que nem consegui t...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

O produto escolhido para análise é a série La Casa de Papel da Netflix e Vancouver Media. 

Classificamos como relevantes tweets que mostravam a opinião e sentimentos, sejam eles positivos ou negativos, sobre a série.
*Consideramos tweets relacionados a spoilers como relevantes pois eles também demonstram um forte sentimento que as pessoas possuem em relação à série.

Classificamos como não relevantes tweets que não se encaixaram na nossa classificação de relevância, tweets que falavam sobre tópicos pessoais ou tweets que falavam sobre algum personagem específico da série.


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [22]:
filtro_nao_relevante = train['Classificação (relevante = 1, não relevante = 0)']==0
filtro_relevante = train['Classificação (relevante = 1, não relevante = 0)']==1


In [23]:
relevantes_train = train[filtro_relevante]
nao_relevantes_train = train[filtro_nao_relevante]

In [24]:
relevantes_train

,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0
3,la casa de papel me faz torcer pros bandidos,1.0
4,la casa de papel acabou comigo me encontro des...,1.0
5,nossa mas que inferno esse final de lá casa de...,1.0
6,vey chorei em vis a via igual uma criança agor...,1.0
...,...,...
291,eu não chorei vendo lá casa de papel não né ro...,1.0
296,eu achei que nenhuma morte de la casa de papel...,1.0
297,caralho eu to chorando igual uma vagabunda com...,1.0
298,cara eu nunca chorei tanto quanto eu chorei ol...,1.0


In [25]:
nao_relevantes_train

,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
1,agora vou assistir lá casa de papel e dormir p...,0.0
2,dedo coçando pra assistir lá casa de papel mas...,0.0
10,denver o maior de la casa de papel e ngm me co...,0.0
12,o rio de lá casa de papel é maior prova de que...,0.0
13,krl vi agora q to vendo a psrte 4 de la casa d...,0.0
...,...,...
290,lucas tem que parar de me dar spoiler de la ca...,0.0
292,faltou só ela pra maratonar la casa de papel cmg,0.0
293,odeio mt essa grávida de la casa de papel q tr...,0.0
294,@flaviagasi se pá tem uns que acham que é la c...,0.0


# Base de Teste

In [26]:
## Salvar tweets como strings para limpar 
relevantes_train_txt = ''
for tweet in relevantes_train['Treinamento']:
    relevantes_train_txt += " "
    relevantes_train_txt += str(tweet)

nao_relevantes_train_txt = ''
for tweet in nao_relevantes_train['Treinamento']:
    nao_relevantes_train_txt += " "
    nao_relevantes_train_txt += str(tweet)

relevantes_train_txt


' tô vendo la casa de papel mdssss o primeiro ep já foi de tirar o fôlego la casa de papel me faz torcer pros bandidos la casa de papel acabou comigo me encontro desidratada de tanto chorar! nossa mas que inferno esse final de lá casa de papel vey chorei em vis a via igual uma criança agora em lá de casa de papel pqp! eu to chorando com esse último episódio de la casa de papel mano rosto_chorando_aos_berros esperei uma eternidade pra sair a 5° temporada de la casa de papel pra assistir em 1 dia kkkk e vamos de esperar mais uma vida p 6° temporada rosto_desolado la casa de papel vai se fude que odio meu deus nunca mais vou assistir lá casa de papel obrigado la casa de papel por me fazer chorar sem parar novamente como que cagaram no pau com la casa de papel assim mdeus tava bom so com 3 temporadas assisti la casa de papel q lixo pqp falta só um ep da lá casa de papel pra acabar e eu não queria acabar agr pq eu já peguei um spoiler da tokio que eu não vou aceitar rosto_chorando a morte d

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**